In [1]:
from transformers import AutoModelForMaskedLM , AutoTokenizer
import pandas as pd
import numpy as np
import torch
model_path=r"E:\code\bert-base-chinese"
model = AutoModelForMaskedLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

d:\ProgramData\Anaconda3\envs\torch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at E:\code\bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
#1、加载数据
train_df = pd.read_csv(r'data.csv', encoding='utf-8', header=None, names=['label','review'])
print(train_df.shape)

sentences = list(train_df['review'][1:])
labels =train_df['label'][1:].values
labels = [int(i) for i in labels]
print(sentences[0:5])
print(labels[0:5])

(1243, 2)
['商业秘密的秘密性那是维系其商业价值和垄断地位的前提条件之一', '南口阿玛施新春第一批限量春装到店啦\ue310   \ue310   \ue310   春暖花开淑女裙、冰蓝色公主衫\ue006   气质粉小西装、冰丝女王长半裙、\ue319   皇', '带给我们大常州一场壮观的视觉盛宴', '有原因不明的泌尿系统结石等', '23年从盐城拉回来的麻麻的嫁妆']
[0, 1, 0, 0, 0]


In [3]:
from collections import Counter
c = Counter(labels)
print (dict(c))

{0: 1120, 1: 122}


In [4]:
prompt="这是一条垃圾短信吗？答案是[MASK]。"
goodToken="是"
badToken="否"
good=tokenizer.convert_tokens_to_ids(goodToken)
bad=tokenizer.convert_tokens_to_ids(badToken)
print(good)
print(bad)

3221
1415


In [5]:
#获取gpu和cpu的设备信息
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device=",device)
if torch.cuda.device_count()>1:
    print("Let's use ",torch.cuda.device_count(),"GPUs!")
    model=nn.DataParallel(model)
model.to(device)

device= cuda


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [6]:
#batch finetune
from torch.utils.data import Dataset,DataLoader,random_split

class DataToDataset(Dataset):
    def __init__(self,sentences,labels):
        self.sentences = [sen + prompt for sen in sentences]
        self.tokens_tensor = tokenizer(self.sentences, return_tensors='pt', padding=True)['input_ids']
        self.mask_pos = [torch.nonzero(item==103).squeeze() for item in self.tokens_tensor]
        self.labels = torch.tensor(labels)
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self,index):
        return self.tokens_tensor[index],self.mask_pos[index],self.labels[index]

#封装数据

datasets=DataToDataset(sentences,labels)
train_size=int(len(datasets)*0.1) #仅用10%数据的训练
test_size=len(datasets)-train_size
print([train_size,test_size])
train_dataset,val_dataset=random_split(dataset=datasets,lengths=[train_size,test_size])

[124, 1118]


In [7]:
BATCH_SIZE=4
#这里的num_workers要大于0
train_loader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
val_loader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)#

In [8]:
#train model
lossFunc=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-5)

from sklearn.metrics import accuracy_score
def flat_accuracy(preds,labels):
    pred_flat=np.argmax(preds,axis=1).flatten()
    labels_flat=labels.flatten()
    return accuracy_score(labels_flat,pred_flat)

epochs=3
for epoch in range(epochs):
    train_loss = 0.0
    train_acc=0.0
    for i,data in enumerate(train_loader):
        tokens_tensor,mask_pos,labels=[elem.to(device) for elem in data]
        #优化器置零
        optimizer.zero_grad()
        #得到模型的结果
        outputs = model(tokens_tensor.cuda(device))
        predictions = outputs[0]
        pred = torch.zeros(len(mask_pos), 2)
        for inum in range(len(mask_pos)):
            #print(mask_pos[inum])
            pred[inum] = predictions[inum, mask_pos[inum]][[bad,good]]
        #计算误差
        loss=lossFunc(pred.to(device), labels)
        train_loss += loss.item()
        #误差反向传播
        loss.backward()
        #更新模型参数
        optimizer.step()
        #计算acc 
        #out=pred.detach().numpy()
        pred=pred.detach().cpu().numpy()
        #labels=labels.detach().numpy()
        labels=labels.detach().cpu().numpy()
        train_acc+=flat_accuracy(pred,labels)
        if (i + 1) % 10 == 0:
            print("train %d/%d epochs Batch %d Loss:%f, Acc:%f" %(epoch+1,epochs, (i+1), train_loss/(i+1),train_acc/(i+1)))
    print("train %d/%d epochs Loss:%f, Acc:%f" %(epoch+1,epochs,train_loss/(i+1),train_acc/(i+1)))

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


train 1/3 epochs Batch 10 Loss:0.622141, Acc:0.950000
train 1/3 epochs Batch 20 Loss:0.419353, Acc:0.950000
train 1/3 epochs Batch 30 Loss:0.348910, Acc:0.933333
train 1/3 epochs Loss:0.337670, Acc:0.935484
train 2/3 epochs Batch 10 Loss:0.102772, Acc:0.950000
train 2/3 epochs Batch 20 Loss:0.065760, Acc:0.975000
train 2/3 epochs Batch 30 Loss:0.045607, Acc:0.983333
train 2/3 epochs Loss:0.044241, Acc:0.983871
train 3/3 epochs Batch 10 Loss:0.003213, Acc:1.000000
train 3/3 epochs Batch 20 Loss:0.002806, Acc:1.000000
train 3/3 epochs Batch 30 Loss:0.002026, Acc:1.000000
train 3/3 epochs Loss:0.002212, Acc:1.000000


In [20]:
#batch evaluate
from sklearn import metrics

print("evaluate...")
pred_list = []
y_list = []
model.eval()
for j,batch in enumerate(val_loader):
    tokens_tensor,mask_pos,val_labels=[elem.to(device) for elem in batch]
    with torch.no_grad():
        outputs = model(tokens_tensor.cuda(device))
        predictions = outputs[0]
        pred = torch.zeros(len(mask_pos), 2)
        for inum in range(len(mask_pos)):
            pred[inum] = predictions[inum, mask_pos[inum]][[bad,good]]
        pred=pred.detach().cpu().numpy()
        pred_flat=np.argmax(pred,axis=1).flatten()
        pred_list.extend(pred_flat)
        val_labels=val_labels.detach().cpu().numpy()
        y_list.extend(val_labels)
    
classify_report = metrics.classification_report(pred_list, y_list, digits = 4) #分类报告 support测试集样本数
print(classify_report) 
confusion_matrix = metrics.confusion_matrix(pred_list, y_list) #混淆矩阵
print(confusion_matrix) 

evaluate...
              precision    recall  f1-score   support

           0     0.9931    0.9862    0.9897      1018
           1     0.8692    0.9300    0.8986       100

    accuracy                         0.9812      1118
   macro avg     0.9311    0.9581    0.9441      1118
weighted avg     0.9820    0.9812    0.9815      1118

[[1004   14]
 [   7   93]]
